## <CENTER> Fitting functions to distributions in ROOT

In [ ]:
# import several types of objects we'll need (the ones starting with k represent predefined colors)
from ROOT import TH1F, TCanvas, TF1, TLegend, kRed, kBlue, kGreen, kBlack

###  2.1 Histograms, functions and uncertainties

In this section you'll familiarize yourselves with the ROOT classes for
* drawing distributions in [histograms](https://en.wikipedia.org/wiki/Histogram): see [the documentation for the TH1 classes](https://root.cern.ch/doc/master/classTH1.html)
* describing mathematical functions: see [the documentation for the TF1 class](https://root.cern.ch/doc/master/classTF1.html)
________________________

In [ ]:
# create a histogram with object name "histo", title and axis labels as listed, 
# containing 20 bins in the range 0 to 3
histo = TH1F("histo", "Events; x; Number of random numbers", 20, 0., 3.)

# create a function that can be used to generate random numbers, in this case we use an exponential function
myFunctionName = "myExponential"
myFunction = TF1(myFunctionName, "exp(-2.7*x)", 0, 3) # defined on interval 0 < x < 3
myFunction.Print() # prints out some info about the function object, including the actual expression

In [ ]:
# number of random numbers to generate
N = 500

# fill the histogram with random values following a distribution defined by the function
histo.FillRandom(myFunctionName, N)

# check how many entries were filled into the histogram
print("Number of entries: %d" % histo.GetEntries())

In [ ]:
# create a canvas object that we can draw things on
canvas = TCanvas("MyCanvas", "Test fits", 800, 600)

# draw the histogram
histo.Draw("e1")
canvas.Draw()

#### Note that each point is drawn with a vertical bar representing the uncertainty of the value. Find out what this uncertainty comes from. The bar in the x-direction shows the size of each bin and does not correspond to an uncertainty in the same way.

**Hint:** Hold the cursor over a data point to see its value and the associated uncertainty.

____________________________________________________________________________________

### 2.2 Make a fit
Now let's fit a function to the data in the histogram. The function used is `TH1F::Fit(func, fit_opt, draw_opt, xlow, xup)` where
* `func` is either a string with the internal ROOT object name of a function (i.e. a `ROOT::TF1` object), like `myExponential` in the example above, or a function object itself, like `myFunction` above. 
* `fit_opt` is a string which describes the fit options to use (e.g. `fit_opt = "V"` for verbose), and `draw_opt` defines the draw options (e.g. `draw_opt="E"` to draw the histogram with errors). Several options can be given simultaneously by combining several letters into a string.
* `x_low` and `x_up` define x-axis range to use for the fit. 

See further info in [the documentation for TH1::Fit()](https://root.cern.ch/doc/master/classTH1.html#a7e7d34c91d5ebab4fc9bba3ca47dabdd). You can read e.g. about the different fit options to control the fit (more on this soon).

Here we'll try to fit the same function we used to fill the histogram (should by definition work well).
By calling `histo.Fit()` the fit is performed and the resulting curve is drawn in the canvas on top of the histogram.
By default a weighted least-squares fit is used, also denoted $\chi^2$-fit.
___________________________________________

#### Side note: how `TF1` functions can be defined

As we saw above, a function can be defined by entering the corresponding mathematical expression as a string. Free parameters can be defined in with placeholders in square brackets, e.g. an arbitrary second-degree polynominal can be be specified with the following line of code, where `[n]` represents parameter `n`:

In [ ]:
functionWithParameters = TF1("myFunction", "[0] + [1]*x + [2]*x^2")
functionWithParameters.Print()

The free parameters can the be assigned (starting) values:

In [ ]:
# sets all parameters, i.e. 0, 1 and 2, in that order
print("Will set all parameters")
functionWithParameters.SetParameters(4.3, -2, 0.3) 
for p in range(functionWithParameters.GetNpar()):
    print("Parameter %d = %.2f" % (p, functionWithParameters.GetParameter(p)))

# set the value of the 2nd parameter (index = 1) to 10
print("Will now set only parameter 1")
functionWithParameters.SetParameter(1, 10) 
for p in range(functionWithParameters.GetNpar()):
    print("Parameter %d = %.2f" % (p, functionWithParameters.GetParameter(p)))

There are two other ways to define functions that can come in handy (especially the latter):
* Use predefined functions through reserved keywords for the name, e.g. `expo` (exponential), `gaus` (for a Gauss distribution), etc (see below)
* Use the functions defined in `TMath` (see [documentation](https://root.cern.ch/doc/master/namespaceTMath.html)), e.g.:

In [ ]:
fMin = TF1("myFunction", "TMath::Min(x, 3.1)", 0, 10)
fMin.Draw("l")
canvas.Draw()

In [ ]:
fMin.Eval(4.3) # evaluate for a given value of x (try putting in other values, does it behave like you expect?)

In [ ]:
# now make a different version of this function, one that has a free parameter for the cut-off value
fMin2 = TF1("myFunc", "TMath::Min(x, [threshold])", 0, 10) # the free parameter "threshold" is defined
fMin2.Print()

print("By default unspecified parameters get the value %f" % fMin2.GetParameter(0))

# set the value of the parameter to what you want, here we try 1.7 just to change something
fMin2.SetParameter(0, 1.7)
fMin2.SetLineColor(3) # let's draw this second one with a green line
fMin2.Draw("SAME")
canvas.Draw()

In [ ]:
# let's try a Landau distribution, with free parameters for the most probable value and the width
fLandau = TF1("landauFunction", "TMath::Landau(x, [MPV], [width])", 0, 10)
# now let's give the parameters values: MPV of 4.5 and a width of 0.7
fLandau.SetParameter("MPV", 4.5)
fLandau.SetParameter("width", 0.7)
fLandau.Draw()
canvas.Draw()
fLandau.Print()

#### Back to making a fit

Let us now fit an exponential function with free parameters to the histogram we generated and see if we can determine the original function.

In [ ]:
# "expo" used here is shorthand for "exp([0]+[1]*x)"
fitFunction = TF1("myFitFunction", "expo", 0, 3)
fitFunction.Print()

# this is where the magic happens, ROOT adjusts the free parameters of the function to the values
# that make it best describe the distribution in the histogram
fitresult = histo.Fit(fitFunction, "S") # option "S" makes sure the fit result is returned

canvas.Draw() 

Above the figure the parameters result of the fit is printed. Note in particular the `STATUS` which shows if the fit converged (found a minimum) or not. The best-fit values of the parameters and their corresponding uncertainties are printed. By passing fit option `"V"` more verbose output will be given.

Print out the relative uncertainty on the `Slope` parameter below!

Hint: `fitresult` is an object of type `TFitResult`, and you can access the value for parameter `i` by calling the function `Parameter(i)` on this object. Check out [the documentation for TFitResult](https://root.cern.ch/doc/master/classROOT_1_1Fit_1_1FitResult.html) to find the function which returns the uncertainty on the parameter values that the fit converged on.

In [ ]:
# print("Relative uncertainty Slope = ...."

#### Now go back and increase the number of entries in the histogram by a factor of two and rerun the fit. What happens to the uncertainty? What if you increase by a factor of four? Explain.

___________________________________________________________________________

### 2.3   Goodness-of-fit: how well does the model fit the data?
How can we quantify how good a fit is? 
One way is to use the fact that the least-squares sum follows a $\chi^2$ distribution, provided certain conditions are fulfilled.
The number of degress of freedom $N_\rm{DoF}$ is equal to the number of data points (bins) minus the number of parameters to be determined.
With this, the probability to get the observed value for the sum can be evaluated.
With enough bins, one can consider the ratio between the $\chi^2$ sum and the number of degrees of freedom. 
Since each term in the sum should on average contribute around one, the ratio should be close to one for a good fit. 
The ratio is called the "reduced chi-square".
Use the measure with care: the $\chi^2$ assumption only holds if the model is correct, linear in the parameters, and the data points are approximately Gaussian with uncertainties estimated correctly.

Below we investigate this by making a fit to a square wave form.
A short assignment follows.

In [ ]:
import math
# create a histogram and populate it to look like a square pulse that we can then use to generate random values
squareWave = TH1F("squareWave", "Square; x; y", 3, 0.5, 3.5)
squareWave.SetBinContent(1, 5.); squareWave.SetBinError(1, math.sqrt(5.))
squareWave.SetBinContent(2, 15.); squareWave.SetBinError(2, math.sqrt(15.))
squareWave.SetBinContent(3, 5.); squareWave.SetBinError(3, math.sqrt(5.))
squareWave.SetLineColor(kBlack)

# sample the histogram and fill the random values into another histogram
sample_squareWave = TH1F("sample_squareWave", "; x; y", 15, 0.5, 3.5)
n=8000
sample_squareWave.FillRandom(squareWave, n)

# draw the original model and the sampled distribution
sample_squareWave.Draw("e1")
squareWave.Scale(float(n)/((15.+5.+5.)/3)/sample_squareWave.GetNbinsX()) 
squareWave.Draw("hist same")
canvas.Draw()

In [ ]:
# now let's fit a constant function to this distribution, i.e. f = C
const = TF1("const", "[0]", 0.5, 3.5, 1) # the function just has one constant parameter and does not depend on x
const.SetParameter(0, 10.) # set the initial guess to 10
fitresult = sample_squareWave.Fit("const", "S", "SAME", 0.5, 3.5)
print("\n*** Chi-square sum = {:.1f}, Number of degrees of freedom = {}, ratio = {:.1f}".format(fitresult.Chi2(), 
                                                                                                fitresult.Ndf(), 
                                                                                                fitresult.Chi2()/fitresult.Ndf()))

canvas.Draw()

As expected, the fit puts the line somewhere in between the upper and lower part of the pulse.
From the print-out we note that the $\chi^2/N_\rm{DOF}$ is far from one.
The model does not describe the data well.

<b> Now try to fit only a range in which a constant should be a good description of the data.
    Evaluate $\chi^2/N_\rm{DOF}$. Is it closer to one? </b>

____________________________________________________________________________
### 2.4 Configuring the fit
By default, the fit is done by minimizing the sum of weighted least-squares ($\chi^2$). This is based on the assumption that the measured values come from a Gaussian distribution. In our case, they are counts, and come from the Poisson distribution. But from courses in statistics we know that the Poisson distribution becomes increasingly Gaussian-like with increasing expectation value. 

You can give the fit option `"L"` to instead make a maximum likelihood fit. In this case, a likelihood function is maximised, where the function is equal to the product of Poisson factors, where each bin contributes with one factor.

Let's compare the two alternatives using an example. Consider a model equal to a straight line. We generate a number of events from the model and draw the histogram, corresponding to the observation made by an experiment. The goal is to measure the model's slope by performing a fit.
_______________________________________________________________________________

In [ ]:
# a simple straight-line model, y = kx + m
xlow = 3.; xup = 10.
fmodel = TF1("lin", "[0]*x+[1]", xlow, xup)
# set initial parameter values
fmodel.SetParameters(1., 1.)
# create a histogram
hist = TH1F("exp", "; x; Number of events", 10, xlow, xup)
n = 30
hist.FillRandom("lin", n) # fill histogram with n randomly generated numbers from the function

# fit with the default least squares method
fitresult = hist.Fit("lin", "S", "e1") # store fit results using fit option "S"

# get parameters and their uncertainties
print(fitresult.Parameter(0))
print(fitresult.ParError(0))

# save the function and draw it again in blue (to prevent it from disappearing when drawing the same function again later)
fitresultLeastSquares = hist.GetFunction("lin").Clone("linLeastSquares") # Clone() creates a copy of the function
fitresultLeastSquares.SetLineColor(kBlue)
fitresultLeastSquares.Draw("SAME")
print("\nSlope least squares = {:.3f}".format(fitresultLeastSquares.GetParameter(0)))

# re-run the fit, this time with option "L". The resulting curve is shown in red
hist.Fit("lin", "L", "e1 SAME")
print("Slope likelihood = {:.3f}\n".format(hist.GetFunction("lin").GetParameter(0)))
canvas.Draw()

#### Note that the two fitted lines are not identical. Try what happens if you increase the number of events - do you understand why?
_______________________________________________________________________________

### 2.5 Define your own functions
______________________________________________________________________

In [ ]:
# NB! The interactive feature unfortunately does not work for all functions. 
# We'll turn it off to make sure everything is drawn correctly.
%jsroot off

In [ ]:
# define your own function, starting with a*(x-b)^2

# alternative 1: write the formula as a string using "TFormula"-syntax
# in the below, [i] represent free parameters, the last two arguments 
# are the range for the function
f1 = TF1("f1", "[0]*(x-[1])*(x-[1])", -10., 10.) 

# alternative 2: write with already existing functions (pre-defined eller user defined)
f2 = TF1("f2", "gaus + f1", -10., 10.)

# alternative 3: using your own python function (for info, you likely won't need this)
def myfunc(x, params):
    x = x[0]
    a = params[0]; b = params[1]
    return a*(x-b)**2
f3 = TF1("f3", myfunc, -10., 10., 2) # the last argument specifies the number of parameters

# set parameter values
f1.SetParameters(1., 2.)
f2.SetParameters(100., -2., 4., 1., 2.) # the first three parameters are for the Gaussian, the last two for the f1 function
f3.SetParameters(1., 3.)

# set limits for the parameters (that they are forced to stay inside) Useful to fitting
f1.SetParLimits(0, -3., 5.) # parameter0 must lie inside [-3., 5.] 

# colors for drawing
f1.SetLineColor(kBlue)
f2.SetLineColor(kBlack)
f3.SetLineColor(kRed)

# draw
f1.Draw()
f2.Draw("SAME")
f3.Draw("SAME")
canvas.Draw()

___________________________________________________

### 2.6 Add a legend
In general, figures should always have legends specifying what the different curves/histograms shown in it represent. Below is an example, see [the TLegend documentation](https://root.cern.ch/doc/master/classTLegend.html) for further details.

In [ ]:
### create the legend object
leg = TLegend(0.5, 0.5, 0.8, 0.8) # the arguments are x1, y1, x2, y2 coordinates, given as fractions of the canvas size

### add the curves we drew above to the legend. Syntax is TLegend::AddEntry(drawn_object, title, plot_style)
leg.AddEntry(f1, "Parabola, defined with TFormula", "l") # "l" means "line", i.e. the line of f1 will be shown in the legend
leg.AddEntry(f2, "Parabola + Gaussian", "l")
leg.AddEntry(f3, "Parabola, user defined", "l")

## make legend prettier (see below)
# ...

## draw legend
leg.Draw()
canvas.Draw()

_______________________________________________________________
Make sure the font in the legend is not too small. This can be set with `leg.SetTextSize(x)` where `x` is fraction of canvas size (try something like 0.03). The legend border can be removed with `leg.SetBorderSize(0)`. Change the code block above to make the legend prettier.
_______________________________________________________________

### See if you can answer the following questions - you will need to understand them to properly explain the fit procedure and results for the report.
* In a histogram, if drawn with with some `e` option, data points will be displayed with an uncertainty. ROOT has automatically evaluated the uncertainty to... what?
* What determines the uncertainty on the fitted parameters? How can they be reduced?
* ROOT uses the weighted least squares ($\chi^2$) fit by default. When is this appropriate? What happens if we use the option `"L"`?

Now you're ready to find an example particle in the data, the $Z^0$ boson - gå vidare till [3-TheMassOfTheZ](3-TheMassOfTheZ.ipynb)!